In [1]:
import inspect
import itertools

import numpy as np

import skimage as ski
import skimage.filters.rank as sfr

In [2]:
cell_4d = ski.data.cells3d()
imgs = (cell_4d[:, 0],  # Membranes
        cell_4d[:, 1],  # Nuclei
        ski.data.brain())
img = imgs[0]

In [3]:
img.shape

(60, 256, 256)

In [4]:
footprint = ski.morphology.footprint_rectangle([8, 9, 10])

In [5]:
filt_orig = sfr.equalize(img, footprint)

/Users/mb312/.local/lib/python3.12/site-packages/skimage/filters/rank/generic.py:353: UserWarning: Bad rank filter performance is expected due to a large number of bins (65536), equivalent to an approximate bitdepth of 16.0.
  image, footprint, out, mask, n_bins = _handle_input_3D(


In [6]:
shift_args = ['shift_' + c for c in 'xyz']

def shargs(shifts):
    return dict(zip(shift_args, shifts))

def rolled_filt(img, footprint, filt, axes, shifts=[0, 0, 0]):
    r_img = np.transpose(img, axes)
    r_shifts = np.array(shifts)[list(axes)]
    r_fp = np.transpose(footprint, axes)
    f_r_img = filt(r_img, r_fp, **shargs(r_shifts))
    return np.transpose(f_r_img, np.argsort(axes))

In [7]:
filt_rolled = rolled_filt(img, footprint, sfr.equalize, (2, 1, 0))
assert np.all(filt_rolled == filt_orig)

In [8]:
shifts = [1, 2, 3]
filt_start = sfr.equalize(img, footprint,
                          **shargs(shifts))
filt_s_r = rolled_filt(img, footprint, sfr.equalize, (2, 1, 0),
                       shifts)
assert np.all(filt_start == filt_s_r)

In [9]:
filters = []
for filt in sfr.__dict__.values():
    if not callable(filt):
        continue
    params = list(inspect.signature(filt).parameters)
    if params[1] != 'footprint':
        continue
    if params[4:7] == shift_args:
        filters.append(filt)
filters

[<function skimage.filters.rank.generic.autolevel(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.equalize(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.gradient(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.majority(image, footprint, *, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.maximum(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.mean(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.geometric_mean(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <function skimage.filters.rank.generic.subtract_mean(image, footprint, out=None, mask=None, shift_x=0, shift_y=0, shift_z=0)>,
 <func

In [10]:
orderings = set(itertools.permutations(range(3), 3))
orderings.remove((0, 1, 2))

In [11]:
for filt in filters:
    print(filt.__name__)
    for i, img in enumerate(imgs):
        filt_start = filt(img, footprint, **shargs(shifts))
        print(f'Image {i}')
        for order in orderings:
            print(f'Ordering {order}')
            filt_s_r = rolled_filt(img, footprint, filt,
                                   order,
                                   shifts)
            assert np.all(filt_start == filt_s_r)

autolevel
Image 0
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 1
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)


/Users/mb312/.local/lib/python3.12/site-packages/skimage/filters/rank/generic.py:353: UserWarning: Bad rank filter performance is expected due to a large number of bins (51077), equivalent to an approximate bitdepth of 15.6.
  image, footprint, out, mask, n_bins = _handle_input_3D(


Image 2
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
equalize
Image 0
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 1
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 2
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
gradient
Image 0
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 1
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 2
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
majority
Image 0
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 1
Ordering (0, 2, 1)
Ordering (1, 2, 0)
Ordering (2, 1, 0)
Ordering (2, 0, 1)
Ordering (1, 0, 2)
Image 2
Ordering (0, 2, 1)
Ordering (1, 2, 0)
